In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#


# Wire Antenna Basics:  Design Optimization

## Optimizing for minimum peak VSWR within a band of interest

This page demonstrates the most basic sort of antenna optimization: adjusting an antenna's dimensions to produce the lowest peak VSWR within some band of frequencies. For the moment we won't be taking into account any matching schemes, just direct feedline connection to the antenna. Some examples using series section matching, for example, will be presented in subsequent files.

We'll be using the excellent Pymoo optimization library [https://pymoo.org](https://pymoo.org).

This library contains a host of optimization algorithms, but at the moment we'll only be using a simple single-objective genetic algorithm. Elsewhere on this site we'll have examples of more advanced tasks, e.g. multi-objective optimization to explore trade-offs between forward gain, F/B, and bandwidth in Yagi design.



### Example 1: A Simple Dipole

Find the optimum length for a dipole antenna to minimize the peak VSWR within a frequency range.

Pymoo requires the creation of a Problem class that encapsulates the details of the problem to be solved. You should be able to reuse this code with only minor modification for your own similar designs.



In [ ]:
from pymoo.core.problem import Problem
from necutil import nec5_sim_stdio3


#
# A base class for simple single-objective problems (e.g. minimize dipole vswr within a band)
#
class DipoleSingleOptProblem(Problem):
    
    # 
    def __init__(self, n_var, segs_per_m, radius, f_min, f_max, z, f_num=9, **kwargs):
        super().__init__(n_var=n_var,
                         n_obj=1,
                         n_ieq_constr=0,
                         **kwargs)


        self.f_min = f_min              # Frequency band of interest: min, max, # of points
        self.f_max = f_max
        self.f_num = f_num        
        self.freqs = np.linspace(f_min, f_max, num=f_num)       # Freqs of interest as an array
        self.f_center = np.mean([f_min,f_max])                  # Center freq
        self.z0 = 50                    # TL impedance
        self.z = z                      # z dimension (height of antenna)
        self.segs_per_m = segs_per_m    # NEC secgents per meter 
        self.radius = radius            # Wire radius (m)
        
        # NEC5 design deck template
        # GX: mirror across xz plane
        # EX: feedpoint specified as tag=1, segment=1, near end (1)
        # GD: Some typical MININEC ground parameters
        self.necpre = 'CE Dipole\n'             # Obligatory comment line
        self.necpost = f"""GX 100 010
GE 1 0
GD 0 0 0 0 13 0.005 0 0
EX 4 1 1 1 1.0 0.0
FR 0 {f_num} 0 0 {f_min} {(f_max-f_min)/(f_num-1)}
XQ 0
EN
"""
 
    def _make_nec5_design(self, x):
        design = self._make_design(x)                                # The design as array of endpoints
        nec_str = gen_nec5_str(design, self.segs_per_m, self.radius)     # Convert to a string of NEC cards
        return self.necpre + nec_str + self.necpost                 # Return as complete NEC deck


    # Evaluate the designs in X, an array of shape (population_size, number_of_variables)
    def _evaluate(self, X, out, *args, **kwargs):

        # Customize _make_nec5_design() for your specific problem
        designs = [self._make_nec5_design(x) for x in X]             # Make a list of NEC decks, one per row of X

        res = nec5_sim_stdio3(designs, timelimit=10000.0)           # Run the simulations

        # Our 'XQ' card produces feedpoint impedances that look like:
        #  res[design#][0][0] = 
                    # [[28.0, (18.156-28.716j)],
                    # [28.05, (18.262-26.878j)],
                    # [28.1, (18.338-25.051j)],
                    # [28.15, (18.383-23.23j)],
                    # [28.2, (18.396-21.412j)],
                    # [28.25, (18.377-19.592j)],
                    # [28.3, (18.325-17.767j)],
                    # [28.35, (18.243-15.933j)],
                    # [28.4, (18.129-14.086j)]]

        # Extracts feedpoint complex z for each design --> complex array of shape (#designs, #freqs)
        zs = np.array([[freq[1] for freq in des[0][0]] for des in res])

        # Each row in 'vswr_curves' is the vswr curve across the band for one of the designs
        abs_refl_coef = np.abs((zs - self.z0) / (zs + self.z0))         # Reflection coefs
        vswr_curves = (1 + abs_refl_coef) / (1 - abs_refl_coef)         # Vswr
        max_vswr = np.max(vswr_curves, axis=1)[:,None]                  # Max vswr within band for each design (as column vector)

        # Return the objective value we're minimizing (simply the max vswr in this case)
        out["F"] = max_vswr

        # You can also attach other data to the population of designs
        # We'll save the vswr curves for each
        out["VSWR"] = vswr_curves


